# Subida de los datasets a AWS S3

Hemos decidido subir el dataset a AWS para poder tenerlo guardado ahi ya que son datasets de imágenes y ocupan mucho en disco, para ello hemos creado este cuaderno explicativo de como es tanto la subida como la descarga de los datasets.

## Instalaciones

Tenemos que tener instalado kaggle y boto3 en nuestro ordenador para poder acceder a **Kaggle** y poder descargar los datasets y **boto3** para configurar las claves para subir los datos a S3.

In [ ]:
pip install kaggle boto3

## Imports

Importamos las siguientes librerías para poder hacer uso de la API de kaggle y hacer uso de las claves para AWS.

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
import boto3
import os

## Código

Función de descarga de los dataset, usando la API de kaggle mas las crendenciales nuestras de kaggle nos descarga los modelos usando `api.dataset_download_files()`, le hemos pueso `unzip=True` para que los descomprima al descargarlos y hacerlo mas sencillo.

In [ ]:
def download_dataset(dataset, output_folder):
    """
    Descarga un dataset de Kaggle y lo descomprime en la carpeta indicada.
    """
    api = KaggleApi()
    api.authenticate()
    print(f"Descargando dataset {dataset} en {output_folder}...")
    api.dataset_download_files(dataset, path=output_folder, unzip=True)
    print("Descarga completada.")

Este segundo método lo usamos para subir los dataset de uno en uno a nuestro *bucket* de AWS, **importante las keys** 

In [ ]:
def upload_directory(directory_path, bucket, s3_prefix=""):
    """
    Recorre el directorio indicado y sube todos sus archivos a S3,
    manteniendo la estructura de carpetas.
    """
    aws_id = ''
    aws_key = ''
    aws_token = ''
    region = 'us-east-1'

    # Deja que boto3 use el perfil default de AWS (configurado en ~/.aws/credentials)
    s3_client = boto3.client('s3', aws_access_key_id=aws_id, aws_secret_access_key=aws_key,aws_session_token=aws_token,region_name=region)
    print(f"Subiendo archivos de {directory_path} a s3://{bucket}/{s3_prefix}...")
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, directory_path)
            s3_path = os.path.join(s3_prefix, relative_path)
            print(f"Subiendo {local_path} a s3://{bucket}/{s3_path}...")
            s3_client.upload_file(local_path, bucket, s3_path)
    print("Subida a S3 completada.")

Esta ya es la parte final en la que tenemos un diccionario con los dataset y sus carpetas de detino locales, despues la variable del nombre del *bucket* en nuestro **S3** y ya por ultimo el bucle donde iteramos, descargamos el dataset y seguidamente lo subimos.

In [ ]:
# Diccionario con los datasets y sus carpetas de destino locales
datasets = {
    "gpiosenka/butterfly-images40-species": "butterfly-images40-species",
    "veeralakrishna/butterfly-dataset": "butterfly-dataset",
    "phucthaiv02/butterfly-image-classification": "butterfly-image-classification"
}
    
bucket_name = "dataset-tfm-pgm"  # Usa solo el nombre del bucket, no el ARN

# Itera sobre cada dataset para descargarlo y luego subirlo a S3
for dataset, folder in datasets.items():
    download_dataset(dataset, folder)
    upload_directory(folder, bucket_name, s3_prefix=folder)